In [98]:
import os
import json
import shutil
import iaflow
import importlib
import pickle as pkl
import tensorflow as tf

importlib.reload(iaflow)
IAFlow = iaflow.IAFlow

In [104]:
def custom_builder(input_shape):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Input(input_shape),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
  ])
  return model

params_notifier = { # Parameters for notifier, see documentation https://pypi.org/project/notify-function/#description
  'title': 'Training update',
  'webhook_url': os.environ.get('WEBHOOK_URL'),
  'frequency_epoch': 20 # This will send a notification every 20 epochs, by default it is every epoch
}

all_data = tf.data.Dataset.from_tensor_slices((
  tf.random.uniform([1000, 2]),
  tf.random.uniform([1000, 1])
))
train_ds = all_data.take(5)
val_ds = all_data.skip(5)

In [105]:
# Creation of the IAFlow instance
shutil.rmtree('./models', ignore_errors=True)
ia_maker = IAFlow(
  models_folder='./models',
  val_ds=val_ds,
  train_ds=train_ds,
  params_notifier=params_notifier,
  builder_function=custom_builder
)

In [106]:
model_1_data = ia_maker.add_model(
  model_name='model_1',
  model_params={ 'input_shape': (2, 1) }, # Parameters for builder function
  load_model_params={}, # Parameters for model loading, see documentation of tf.keras.models.load_model
  compile_params={ # Parameters for model compilation, see documentation tf.keras.models.compile
    'optimizer': 'adam', 'loss': 'mse',
    'metrics': ['accuracy']
  },
)
#print(f'Data of model_1: {json.dumps(model_1_data, indent=4)}')

GPU info: /physical_device:GPU:0


Model model_1/2022.07.13_21.01.34 added


In [107]:
print('Show all models:')
ia_maker.show_models()

Show all models:
Models:
  model_1
    2022.07.13_21.01.34
      model_ident: model_1_(2, 1)
      model_params: {'input_shape': (2, 1)}
      compile_params: {'optimizer': 'adam', 'loss': 'mse', 'metrics': ['accuracy']}
      load_model_params: {'filepath': './models/model_1/2022.07.13_21.01.34/model_1_(2, 1)_checkpoint.h5'}
      check_path: ./models/model_1/2022.07.13_21.01.34/model_1_(2, 1)_checkpoint.h5
      path_model: ./models/model_1/2022.07.13_21.01.34
      log_dir: ./models/model_1/2022.07.13_21.01.34/logs


In [108]:
ia_maker.train(
  model_1_data,
  batch=32,
  epochs=5
)

Force creation is False. Deleting old logs and model
Creating model
Epoch 1/5


2022-07-13 21:01:48.179464: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.1004 - accuracy: 0.0000e+00

2022-07-13 21:01:48.573228: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step - loss: 0.1004 - accuracy: 0.0000e+00 - val_loss: 0.0843 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 226ms/step - loss: 0.1003 - accuracy: 0.0000e+00 - val_loss: 0.0843 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 236ms/step - loss: 0.1002 - accuracy: 0.0000e+00 - val_loss: 0.0844 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 237ms/step - loss: 0.1001 - accuracy: 0.0000e+00 - val_loss: 0.0843 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 248ms/step - loss: 0.0999 - accuracy: 0.0000e+00 - val_loss: 0.0843 - val_accuracy: 0.0000e+00
Training time: 2.0940680503845215


In [109]:
ia_maker.clear_session()

print('Loading model trained')
ia_maker.train(
  model_1_data,
  batch=32,
  epochs=5,
  val_ds=val_ds,
  train_ds=train_ds
)

Loading model trained
Loading model from ./models/model_1/2022.07.13_21.01.34/model_1_(2, 1)_checkpoint.h5
Epoch 1/5
1/1 [==============================] - ETA: 0s - loss: 0.0998 - accuracy: 0.0000e+00

2022-07-13 21:01:55.320181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-13 21:01:55.528123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 699ms/step - loss: 0.0998 - accuracy: 0.0000e+00 - val_loss: 0.0845 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 431ms/step - loss: 0.0996 - accuracy: 0.0000e+00 - val_loss: 0.0845 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 410ms/step - loss: 0.0995 - accuracy: 0.0000e+00 - val_loss: 0.0845 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 336ms/step - loss: 0.0993 - accuracy: 0.0000e+00 - val_loss: 0.0846 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 276ms/step - loss: 0.0992 - accuracy: 0.0000e+00 - val_loss: 0.0847 - val_accuracy: 0.0000e+00
Training time: 2.171844959259033


In [110]:
ia_maker.delete_model(model_1_data, delete_folder=True)

Model model_1/2022.07.13_21.01.34 deleted
